In [1]:
!pip install transformers

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
# from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig, TFDistilBertModel, DistilBertTokenizer, DistilBertConfig
from google.colab import drive
drive.mount('/drive')

     |████████████████████████████████| 3.1 MB 4.0 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 36.9 MB/s 
     |████████████████████████████████| 895 kB 71.7 MB/s 
     |████████████████████████████████| 3.3 MB 44.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Mounted at /drive


In [2]:
df_reviews = pd.read_json('/drive/MyDrive/Colab Notebooks/CS3244 Machine Learning/CS3244 Group Project/Data/IMDB_reviews.json', lines=True)

In [3]:
print(df_reviews['is_spoiler'])

0          True
1          True
2          True
3          True
4          True
          ...  
573908    False
573909    False
573910    False
573911    False
573912    False
Name: is_spoiler, Length: 573913, dtype: bool


In [4]:
# loading the distilBERT tokenizer and model

dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

# preparing model input
max_len = 32
sentences = df_reviews['review_summary']
labels = df_reviews['is_spoiler']
len(sentences), len(labels)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


(573913, 573913)

In [5]:
dbert_tokenizer.tokenize(sentences[0])


['a',
 'classic',
 'piece',
 'of',
 'un',
 '##for',
 '##get',
 '##table',
 'film',
 '-',
 'making',
 '.']

In [6]:
dbert_inp = dbert_tokenizer.encode_plus(sentences[0], add_special_tokens=True, max_length=20, padding='max_length', truncation=True)
print(dbert_inp)

{'input_ids': [101, 1037, 4438, 3538, 1997, 4895, 29278, 18150, 10880, 2143, 1011, 2437, 1012, 102, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}


In [7]:
# model outputs

id_inp = np.asarray(dbert_inp['input_ids'])
mask_inp = np.asarray(dbert_inp['attention_mask'])
out = dbert_model([id_inp.reshape(1,-1), mask_inp.reshape(1,-1)])
type(out), out

(transformers.modeling_tf_outputs.TFBaseModelOutput,
 TFBaseModelOutput([('last_hidden_state',
                     <tf.Tensor: shape=(1, 20, 768), dtype=float32, numpy=
                     array([[[-0.12724836, -0.13405044, -0.08435404, ..., -0.05197578,
                               0.39620113,  0.16418284],
                             [-0.3032376 , -0.04324305, -0.10760076, ..., -0.00913706,
                               0.58967286,  0.21374361],
                             [ 0.07364839,  0.08216909, -0.09972652, ..., -0.21998073,
                               0.20868465, -0.05606366],
                             ...,
                             [ 0.10738726, -0.19669618, -0.05665012, ...,  0.19439504,
                               0.13160473, -0.00472483],
                             [ 0.06156566, -0.17841981, -0.0741983 , ...,  0.12427461,
                               0.14385164,  0.01551408],
                             [ 0.06906652, -0.0697138 ,  0.0127484 , ..., -0

In [ ]:
# embeddings of a sentence from the output

out[0][:,0,:]


<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-1.27248347e-01, -1.34050354e-01, -8.43539089e-02,
        -1.45718709e-01,  7.97824711e-02, -2.02073336e-01,
         4.01045829e-02,  3.82241100e-01, -1.08997390e-01,
        -6.83179125e-02,  6.15919232e-02, -8.47413093e-02,
        -8.82929862e-02,  5.09693861e-01,  1.01877548e-01,
         5.74164167e-02, -2.57397860e-01,  3.22973356e-02,
         1.89835489e-01, -7.44015425e-02, -9.81273055e-02,
        -3.97422016e-01, -8.14420879e-02,  5.32128587e-02,
        -1.75693437e-01, -3.78673226e-02, -1.53253317e-01,
         1.50326304e-02, -7.31282681e-03,  1.31712228e-01,
         3.95578854e-02, -7.74436444e-02, -1.48689494e-01,
        -1.49230868e-01,  1.35186881e-01, -1.45327345e-01,
         1.49938807e-01, -8.47353786e-02, -9.86596793e-02,
         7.39538595e-02,  1.54541135e-02,  2.11624324e-01,
         2.89556235e-01,  7.16902614e-02, -1.19115405e-01,
        -5.24633974e-02, -2.12862420e+00, -4.20107245e-02,
      

In [8]:
dbert_tokenizer.decode(dbert_inp['input_ids'])


'[CLS] a classic piece of unforgettable film - making. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [9]:
# create a basic NN model using the distilBERT embeddings to get the predictions

num_classes = 2

def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks = Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout = Dropout(0.5)(dense)
    pred = Dense(num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model


model = create_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_1[0][0]',                
 BertModel)                     ast_hidden_state=(N               'input_2[0][0]']                
                                one, 32, 768),                                                    
                                 hidden_states=None                                           

In [10]:
# model input

input_ids = []
attention_masks = []

# tokenise with distilBERT tokeniser
for sent in sentences:
    dbert_inps = dbert_tokenizer.encode_plus(sent, add_special_tokens=True, max_length=max_len, padding='max_length', return_attention_mask=True, truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids = np.asarray(input_ids)
attention_masks = np.array(attention_masks)
labels = np.array(labels)

len(input_ids), len(attention_masks), len(labels)


(573913, 573913, 573913)

In [11]:
# # save model input 

# print('Preparing the pickle file.....')

# pickle_inp_path = 'dbert_inp.pkl'
# pickle_mask_path = 'dbert_mask.pkl'
# pickle_label_path = 'dbert_label.pkl'

# pickle.dump((input_ids), open(pickle_inp_path, 'wb'))
# pickle.dump((attention_masks), open(pickle_mask_path, 'wb'))
# pickle.dump((labels), open(pickle_label_path, 'wb'))


# print('Pickle files saved as ', pickle_inp_path, pickle_mask_path, pickle_label_path)

Preparing the pickle file.....
Pickle files saved as  dbert_inp.pkl dbert_mask.pkl dbert_label.pkl


In [12]:
# # load the input files

# print('Loading the saved pickle files..')

# input_ids = pickle.load(open(pickle_inp_path, 'rb'))
# attention_masks = pickle.load(open(pickle_mask_path, 'rb'))
# labels = pickle.load(open(pickle_label_path, 'rb'))

# print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (573913, 32) Attention mask shape (573913, 32) Input label shape (573913,)


In [13]:
label_class_dict = {0:'True', 1:'False'}
target_names = label_class_dict.values()

In [14]:
# train test split, set up loss function, metric (accuracy) and optimiser (Adam)

train_inp, val_inp, train_label ,val_label, train_mask, val_mask = train_test_split(input_ids, labels, attention_masks, test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

log_dir = 'dbert_model'
model_save_path = './dbert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path, save_weights_only=True, monitor='val_loss', mode='min', save_best_only=True), keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model.compile(loss=loss, optimizer=optimizer, metrics=[metric])

Train inp shape (459130, 32) Val input shape (114783, 32)
Train label shape (459130,) Val label shape (114783,)
Train attention mask shape (459130, 32) Val attention mask shape (114783, 32)


In [ ]:
# # [print(i.shape, i.dtype) for i in train_inp]
# train_inp.dtype

# # x = np.asarray(x).astype(np.float32)
# print(train_inp)
# train_inp = np.asarray(train_inp).astype('float32')
# # train_mask = np.asarray(train_mask)
# # train_label = np.asarray(train_label)
# # val_inp = np.asarray(val_inp)
# # val_mask = np.asarray(val_mask)
# # val_label = np.asarray(val_label)

history = model.fit([train_inp, train_mask], train_label, batch_size=16, epochs=1, validation_data=([val_inp, val_mask], val_label), callbacks=callbacks)



/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


  152/28696 [..............................] - ETA: 23:57:51 - loss: 6.4196 - accuracy: 0.7146